In [1]:
import numpy as np
import pandas as pd

In [5]:
df_users = pd.read_csv('users.csv')
df_users.shape

(337, 11)

In [6]:
df_repositiories = pd.read_csv('repositories.csv')
df_repositiories.shape

(27172, 9)

## Q1 Who are the top 5 users in Barcelona with the highest number of followers? List their login in order, comma-separated

In [13]:
top_logins = df_users.sort_values(by='followers', ascending=False).head(5)['login'].to_list()
top_logins

['midudev', 'ai', 'raysan5', 'vfarcic', 'spite']

## Q2  Who are the 5 earliest registered GitHub users in Barcelona? List their login in ascending order of created_at, comma-separated.

In [14]:
early_reg = df_users.sort_values(by='created_at').head(5)['login'].to_list()
early_reg

['oleganza', 'gravityblast', 'fesplugas', 'fxn', 'pauek']

## 3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [15]:
import csv
from collections import Counter

# Define the list to store license names
licenses = []

# Read the CSV file with UTF-8 encoding
with open('repositories.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Check if the license_name field is present and not empty
        license_name = row.get('license_name', '').strip()
        if license_name:
            licenses.append(license_name)

# Count the occurrence of each license
license_counts = Counter(licenses)

# Get the 3 most common licenses
top_3_licenses = [license for license, count in license_counts.most_common(3)]

# Print the result as a comma-separated list
print(','.join(top_3_licenses))


null,MIT License,Apache License 2.0


## 4. Which company do the majority of these developers work at?

In [16]:
companies = []

# Read the CSV file with UTF-8 encoding
with open('users.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Get and clean up the company field (ignore empty values)
        company = row.get('company', '').strip()
        if company:
            companies.append(company)

# Count the occurrence of each company
company_counts = Counter(companies)

# Find the most common company
most_common_company = company_counts.most_common(5)

# Print the result
if most_common_company:
    print(most_common_company[0][0])
else:
    print("No company data found.")

None


In [58]:
most_common_company

[('None', 129),
 ('FREELANCE', 4),
 ('RED HAT', 3),
 ('APPLE', 3),
 ('ADEVINTA', 3)]

##  5. Which programming language is most popular among these users?

In [18]:
df_repositiories.language.value_counts().head(1)

,count
language,
JavaScript,5736


## 6. Which programming language is the second most popular among users who joined after 2020?

In [20]:
lang = df_repositiories.loc[df_repositiories['created_at'] > '2020-12-31']
lang.language.value_counts().head(3)

,count
language,
JavaScript,937
TypeScript,596
Python,533


## 7. Which language has the highest average number of stars per repository?

In [21]:
lang = df_repositiories.groupby('language')['stargazers_count'].mean().sort_values(ascending=False)
lang.head(1)

,stargazers_count
language,
Vim Script,3839.625


## 8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [23]:
leader_strength = df_users['followers'] / (1 + df_users['following'])
df_users['leader_strength'] = leader_strength
top_5_users = df_users.sort_values(by='leader_strength', ascending=False).head(5)['login'].to_list()
top_5_users

['midudev', 'vfarcic', 'spite', 'amix', 'cfenollosa']

## 9. What is the correlation between the number of followers and the number of public repositories among users in Barcelona?

In [25]:
df_users[['followers', 'public_repos']].corr()


,followers,public_repos
followers,1.000000,0.071207
public_repos,0.071207,1.000000


## 10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [26]:
from sklearn.linear_model import LinearRegression
x = df_users['public_repos'].values.reshape(-1, 1)
y = df_users['followers']
model = LinearRegression()
model.fit(x, y)
additional_followers_per_repo = model.coef_[0]
additional_followers_per_repo

1.0307580135902696

## 11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

In [27]:
df_repositiories[['has_projects', 'has_wiki']].corr()

,has_projects,has_wiki
has_projects,1.000000,0.322876
has_wiki,0.322876,1.000000


## 12. Do hireable users follow more people than those who are not hireable?

Average of following per user for hireable=true minus the average following for the rest

In [63]:
df_users.groupby('hireable')['following'].mean()

,following
hireable,
True,202.581602


## 13. Some developers write long bios. Does that help them get more followers? What's the impact of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)

In [34]:
# prompt: Some developers write long bios. Does that help them get more followers? What's the impact of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)

# Calculate bio length and handle missing bios
df_users['bio_length'] = df_users['bio'].astype(str).apply(lambda x: len(x.split()))
df_users_with_bio = df_users[df_users['bio_length'] > 0]

# Calculate correlation
correlation = df_users_with_bio['bio_length'].corr(df_users_with_bio['followers'])
print(correlation)

# Linear Regression
x = df_users_with_bio['bio_length'].values.reshape(-1, 1)
y = df_users_with_bio['followers']
model = LinearRegression()
model.fit(x, y)
additional_followers_per_word = model.coef_[0]
additional_followers_per_word

0.08043025722387877


19.344634639933535

## 14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [49]:
from datetime import datetime
def is_weekend(date):
    if date.weekday() >= 5:
        return True
    else:
        return False

df_repositiories['weekend'] = df_repositiories['created_at'].apply(is_weekend)

In [51]:
weekend_logins = df_repositiories[df_repositiories['weekend'] == True]['login'].value_counts().head().index

In [52]:
weekend_logins

Index(['kinow', 'nilportugues', 'ajsb85', 'vfarcic', 'wlsf82'], dtype='object', name='login')

In [53]:
for login in weekend_logins:
    print(login, end=',')

kinow,nilportugues,ajsb85,vfarcic,wlsf82,

## 15. Do people who are hireable share their email addresses more often?

In [54]:
def analyze_email_sharing(users_csv_path='users.csv'):
    # Read the complete CSV file
    df = pd.read_csv(users_csv_path)

    # Convert email column to boolean (True if email exists, False if NaN or empty)
    df['has_email'] = df['email'].notna() & (df['email'] != '')

    # Calculate for hireable users
    hireable_mask = df['hireable'] == True
    if hireable_mask.any():
        hireable_email_fraction = df[hireable_mask]['has_email'].mean()
    else:
        hireable_email_fraction = 0

    # Calculate for non-hireable users
    non_hireable_mask = df['hireable'] != True
    if non_hireable_mask.any():
        non_hireable_email_fraction = df[non_hireable_mask]['has_email'].mean()
    else:
        non_hireable_email_fraction = 0

    # Calculate difference and round to 3 decimal places
    difference = round(hireable_email_fraction - non_hireable_email_fraction, 3)

    # Print debug information
    print(f"Total users: {len(df)}")
    print(f"Hireable users with email: {df[hireable_mask]['has_email'].sum()}/{hireable_mask.sum()}")
    print(f"Non-hireable users with email: {df[non_hireable_mask]['has_email'].sum()}/{non_hireable_mask.sum()}")
    print(f"Hireable fraction: {hireable_email_fraction:.3f}")
    print(f"Non-hireable fraction: {non_hireable_email_fraction:.3f}")

    return difference

# Read and analyze the complete dataset
result = analyze_email_sharing()
print(f"\nFinal result: {result:.3f}")

Total users: 337
Hireable users with email: 67/120
Non-hireable users with email: 100/217
Hireable fraction: 0.558
Non-hireable fraction: 0.461

Final result: 0.098


## 16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

In [55]:
def get_surname(name):
    if name is not None:
        return str(name).strip().split()[-1]
    else:
        return None

In [56]:
df_users['surname'] = df_users['name'].apply(get_surname)

In [57]:
df_users['surname'].value_counts().head(10)

,count
surname,
nan,7
Ortiz,3
Martínez,3
Perez,2
López,2
Ferrer,2
Segura,2
Academy,2
Sanchez,2
